In [ ]:
import json
import pathlib
import pandas as pd
import re

# --- İl plaka eşlemesi ---
PLAKA_TO_IL = {
    1:"Adana",2:"Adıyaman",3:"Afyonkarahisar",4:"Ağrı",5:"Amasya",6:"Ankara",7:"Antalya",8:"Artvin",9:"Aydın",
    10:"Balıkesir",11:"Bilecik",12:"Bingöl",13:"Bitlis",14:"Bolu",15:"Burdur",16:"Bursa",17:"Çanakkale",18:"Çankırı",
    19:"Çorum",20:"Denizli",21:"Diyarbakır",22:"Edirne",23:"Elazığ",24:"Erzincan",25:"Erzurum",26:"Eskişehir",
    27:"Gaziantep",28:"Giresun",29:"Gümüşhane",30:"Hakkâri",31:"Hatay",32:"Isparta",33:"Mersin",34:"İstanbul",
    35:"İzmir",36:"Kars",37:"Kastamonu",38:"Kayseri",39:"Kırklareli",40:"Kırşehir",41:"Kocaeli",42:"Konya",
    43:"Kütahya",44:"Malatya",45:"Manisa",46:"Kahramanmaraş",47:"Mardin",48:"Muğla",49:"Muş",50:"Nevşehir",
    51:"Niğde",52:"Ordu",53:"Rize",54:"Sakarya",55:"Samsun",56:"Siirt",57:"Sinop",58:"Sivas",59:"Tekirdağ",
    60:"Tokat",61:"Trabzon",62:"Tunceli",63:"Şanlıurfa",64:"Uşak",65:"Van",66:"Yozgat",67:"Zonguldak",68:"Aksaray",
    69:"Bayburt",70:"Karaman",71:"Kırıkkale",72:"Batman",73:"Şırnak",74:"Bartın",75:"Ardahan",76:"Iğdır",
    77:"Yalova",78:"Karabük",79:"Kilis",80:"Osmaniye",81:"Düzce"
}

def slugify_tr(s: str) -> str:
    tr_map = str.maketrans({"ç":"c","Ç":"c","ğ":"g","Ğ":"g","ı":"i","İ":"i","ö":"o","Ö":"o","ş":"s","Ş":"s","ü":"u","Ü":"u"})
    s = (s or "").translate(tr_map).lower()
    s = re.sub(r"[^a-z0-9]+", "-", s).strip("-")
    return s or "cikti"

def to_number_or_none(x):
    if x is None:
        return None
    s = str(x).strip()
    if s == "" or s == "-":
        return None
    # binlik noktalari sil, virgulu nokta yap
    s = s.replace(" ", "")
    s = s.replace(".", "")
    s = s.replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None

# --- Klasörler ---
json_dir = pathlib.Path("C:/jsons")
out_dir = pathlib.Path("C:/out")
out_dir.mkdir(parents=True, exist_ok=True)

rows = []
indicator_name = None

# --- JSON'ları oku ve uzun formu kur (ham metin değerlerle) ---
for file in json_dir.glob("*.json"):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)

    if indicator_name is None:
        indicator_name = data.get("gosterge_ad") or "gosterge"

    years = []
    for y in data.get("tarihler", []):
        try:
            years.append(int(y))
        except Exception:
            # yıl başlıkları integer değilse metin olarak bırak
            years.append(str(y))

    for item in data.get("veriler", []):
        try:
            code = int(item["duzeyKodu"])
        except Exception:
            # beklenmedik kod tipleri varsa atla
            continue

        il = PLAKA_TO_IL.get(code, f"Bilinmeyen-{code}")
        values = [str(v) if v is not None else "" for v in item.get("veri", [])]

        # Yıl/değer uzunluğu farklıysa kısa olan kadar eşle
        for year, raw in zip(years, values):
            rows.append({
                "il_kodu": code,
                "il": il,
                "yil": year,
                "deger_raw": raw,                 # JSON’daki haliyle
                "deger_num": to_number_or_none(raw)  # opsiyonel sayısal yorum
            })

# --- DataFrame ve çakışmaları çöz ---
df = pd.DataFrame(rows)

# Aynı il_kodu–yil tekrarı varsa son geleni al
df = df.sort_values(["il_kodu", "yil"]).drop_duplicates(["il_kodu", "yil"], keep="last")

# --- GENİŞ FORM (Ham metin) -> asıl çıktı ---
wide_raw = df.pivot_table(
    index=["il_kodu","il"],
    columns="yil",
    values="deger_raw",
    aggfunc="last"   # metinle uyumlu; ortalama denemez
).reset_index()
wide_raw.columns.name = None
# Yılları artan sıraya diz (sayıysa sayısal, değilse alfabetik)
year_cols = [c for c in wide_raw.columns if c not in ("il_kodu","il")]
try:
    wide_raw = wide_raw[["il_kodu","il"] + sorted(year_cols)]
except Exception:
    pass

# --- (Opsiyonel) GENİŞ FORM (Sayısal) -> istersen bakarsın ---
wide_num = df.pivot_table(
    index=["il_kodu","il"],
    columns="yil",
    values="deger_num",
    aggfunc="last"   # tekrar varsa son değer
).reset_index()
wide_num.columns.name = None
num_year_cols = [c for c in wide_num.columns if c not in ("il_kodu","il")]
try:
    wide_num = wide_num[["il_kodu","il"] + sorted(num_year_cols)]
except Exception:
    pass

# --- Excel'e yaz ---
base = slugify_tr(indicator_name)
xlsx_path = out_dir / f"{base}.xlsx"

with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as writer:
    # 1) Ham metin değerler – JSON’daki GİBİ
    #wide_raw.to_excel(writer, sheet_name="veri_ham", index=False)

    # 2) (Opsiyonel) sayısal yorum – istersen kullan
    wide_num.to_excel(writer, sheet_name="veri_sayisal", index=False)

print(f"Dosya kaydedildi: {xlsx_path}")